In [5]:
# REQUIREMENTS:
# pip install reportlab qrcode pillow

import os
from datetime import datetime
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm
from reportlab.pdfbase import pdfmetrics
from reportlab.lib import colors
import qrcode

def _fit(text, font_name, max_font, min_font, box_w):
    t = "" if text is None else str(text)
    s = max_font
    while s > min_font and pdfmetrics.stringWidth(t, font_name, s) > box_w:
        s -= 0.5
    return max(min_font, s)

def _draw_scaled_title(c, text, page_w, top_y, side_margin,
                       font_bold="Helvetica-Bold", max_font=18, min_font=10):
    box_w = page_w - 2*side_margin
    fs = _fit(text, font_bold, max_font, min_font, box_w)
    c.setFont(font_bold, fs)
    c.setFillColor(colors.black)
    c.drawCentredString(page_w/2, top_y, text)
    return fs

def _qr_image(tmp_path, value, box_mm=16, border=1):
    qr = qrcode.QRCode(version=1, error_correction=qrcode.constants.ERROR_CORRECT_H, box_size=4, border=border)
    qr.add_data(value); qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")
    img.save(tmp_path)
    return box_mm*mm

def _resolve_height(spec, cols, HEIGHTS):
    if isinstance(spec, (int, float)): return float(spec)
    if isinstance(spec, str) and spec in HEIGHTS: return HEIGHTS[spec]
    return HEIGHTS["multi"] if len(cols) == 1 else HEIGHTS["single"]

def _draw_row(c, left, usable_w, top_y, height, cols, pad, fonts, sizes, value_font_max, prefill):
    c.setLineWidth(1.1)
    c.setStrokeColor(colors.black)
    c.rect(left, top_y-height, usable_w, height, stroke=1, fill=0)
    x = left; acc = 0.0
    for i, (field, ratio) in enumerate(cols):
        acc += ratio
        if i < len(cols)-1:
            x_next = left + usable_w * acc
            c.line(x_next, top_y-height, x_next, top_y)
        col_w = usable_w * ratio
        label = field.get("label",""); key = field.get("key","")
        fs = _fit(label, fonts[0], sizes[0], sizes[1], col_w - 2*pad)
        c.setFont(fonts[0], fs); c.setFillColor(colors.black)
        c.drawString(x+pad, top_y - pad - fs*0.2, label)
        val = str(prefill.get(key,""))
        if val:
            vfs = _fit(val, fonts[0], value_font_max, 8, col_w - 2*pad)
            c.setFont(fonts[1], vfs)
            y_val = top_y - max(6.8*mm, min(height-3.0*mm, height*0.55))
            c.drawString(x+pad, y_val, val)
        x = left + usable_w * acc

def _draw_caption(c, text, x, y):
    c.setFont("Helvetica-Bold", 9.0)
    c.setFillColor(colors.black)
    c.drawString(x, y, text)

def _draw_table(c, x, y_top, col_widths_mm, headers, n_rows, row_h_mm):
    col_ws = [wmm*mm for wmm in col_widths_mm]
    total_w = sum(col_ws)
    c.setLineWidth(0.4)
    c.setStrokeColor(colors.black)
    c.rect(x, y_top - (row_h_mm*(n_rows+1)), total_w, row_h_mm*(n_rows+1), stroke=1, fill=0)
    cx = x
    for cw in col_ws[:-1]:
        cx += cw
        c.line(cx, y_top - (row_h_mm*(n_rows+1)), cx, y_top)
    for r in range(n_rows+1):
        yy = y_top - (row_h_mm * r)
        c.line(x, yy, x + total_w, yy)
    c.setFont("Helvetica-Bold", 7.8)
    cur_x = x
    for i, hdr in enumerate(headers):
        c.drawString(cur_x + 1.4*mm, y_top - row_h_mm + 1.8*mm, hdr)
        cur_x += col_ws[i]
    return col_ws

def _draw_table_rows_prefilled_except_last3(c, x, y_top, col_ws, rows, row_h_mm,
                                            empty_cols_idx=(6,7,8), item_no_col_index=0):
    for r in range(len(rows)):
        cur_x = x
        cell_top = y_top - row_h_mm*(r+1)
        for i, cw in enumerate(col_ws):
            avail = cw - 2.2*mm
            text = "" if i in empty_cols_idx else ("" if rows[r][i] is None else str(rows[r][i]))
            fs = 7.4  # uniform across all columns, including Item No.
            c.setFont("Helvetica", fs)
            c.drawString(cur_x + 1.4*mm, cell_top - row_h_mm + 3.2*mm, text)
            cur_x += cw

def render_empty_inventory_from_doc(doc,
                                    filename="inventory_form_a4_empty.pdf",
                                    page_size=A4,
                                    margins_mm=6,
                                    pad_mm=3,
                                    title="INVENTORY – ITEMS RECORD (EMPTY: Add/Sold/Current only)",
                                    base_row_height_mm=4.6*1.2,
                                    qr_mm=16,
                                    qr_border=1,
                                    label_font=("Helvetica","Helvetica-Bold"),
                                    label_font_sizes=(10,7.2),
                                    value_font_max=11.5,
                                    section_gap_mm=6,
                                    title_font=(18,10)):
    w, h = page_size
    m = margins_mm*mm
    pad = pad_mm*mm
    left, right = m, w - m
    usable_w = right - left
    extra_gap = section_gap_mm*mm

    HEIGHTS = {"single":10*mm, "multi":14*mm}
    FIELDS = [
        ([({"key":"form_name","label":"Form Name"},0.32),
          ({"key":"current_form_qr_code","label":"Form QR Code"},0.23),
          ({"key":"generated_on","label":"Generated On"},0.22),
          ({"key":"generated_by","label":"Generated By"},0.23)],"single"),
        ([({"key":"location","label":"Location"},0.50),
          ({"key":"branch","label":"Branch"},0.50)],"single"),
        ([({"key":"date_iso","label":"Date (YYYY-MM-DD)"},0.50),
          ({"key":"date_name","label":"Date (Name e.g., Oct. 12, 2025)"},0.50)],"single"),
    ]

    def _ensure_date_fields(prefill):
        iso = str(prefill.get("date_iso","")).strip()
        name = str(prefill.get("date_name","")).strip()
        if iso and not name:
            try:
                dt = datetime.strptime(iso, "%Y-%m-%d")
                prefill["date_name"] = dt.strftime("%b. %d, %Y")
            except Exception:
                pass
        return prefill

    c = canvas.Canvas(filename, pagesize=page_size)
    c.setTitle(title)

    prefill = {
        "form_name": "Inventory Record (Empty Lines for Add/Sold/Current)",
        "current_form_qr_code": doc.get("current_form_qr_code",""),
        "generated_on": doc.get("date",""),
        "generated_by": "JEF Quickeln",
        "location": doc.get("location",""),
        "branch": doc.get("branch",""),
        "date_iso": doc.get("date",""),
        "cashier_employee_number": "",
        "recorder_employee_number": "",
    }
    prefill = _ensure_date_fields(prefill)

    title_y = h - m - 5*mm
    _draw_scaled_title(c, title, w, title_y, m, font_bold=label_font[1],
                       max_font=title_font[0], min_font=title_font[1])
    cursor = title_y - 3*mm

    qr_value = str(prefill.get("current_form_qr_code","")).strip()
    tmp_qr = None
    if qr_value:
        tmp_qr = os.path.join(os.getcwd(), "inventory_empty_qr.png")
        qr_box = _qr_image(tmp_qr, qr_value, box_mm=qr_mm, border=qr_border)
        x_qr = (w - qr_box)/2
        y_qr = cursor - qr_box - 1.6*mm
        c.drawImage(tmp_qr, x_qr, y_qr, width=qr_box, height=qr_box, preserveAspectRatio=True, mask='auto')
        c.setFont(label_font[0], 7.2)
        c.drawCentredString(w/2, y_qr - 1.6*mm, qr_value)
        cursor = y_qr - 4.5*mm

    for cols, hspec in FIELDS:
        h_row = _resolve_height(hspec, cols, HEIGHTS)
        _draw_row(c, left, usable_w, cursor, h_row, cols, pad,
                  fonts=label_font, sizes=label_font_sizes,
                  value_font_max=value_font_max, prefill=prefill)
        cursor -= h_row

    cursor -= extra_gap

    headers = ["Item No.","Barcode","Product","Unit","Price","Prev Qty","Add","Sold","Current"]
    col_mm  = [12, 34, 62, 14, 16, 16, 14, 12, 18]
    row_h   = base_row_height_mm*mm

    items = list(doc.get("items", []))
    target_rows = len(items) + 2  # exactly 2 extra empty rows beyond the items

    _draw_caption(c, f"INVENTORY ITEMS – Prefilled; Write ONLY Add/Sold/Current ({target_rows} rows)",
                  left, cursor - 1.6*mm)
    cursor -= 6*mm

    col_ws = _draw_table(c, left, cursor, col_mm, headers, target_rows, row_h)

    rows = []
    for idx, it in enumerate(items):
        item_no = it.get("item_number") or str(idx+1)
        rows.append([
            item_no,
            it.get("barcode",""),
            it.get("name",""),
            it.get("unit",""),
            f'{it.get("price","")}',
            f'{it.get("previous_quantity","")}',
            "", "", ""  # Add/Sold/Current left blank
        ])

    while len(rows) < target_rows:
        r_index = len(rows) + 1
        rows.append([str(r_index), "", "", "", "", "", "", "", ""])

    _draw_table_rows_prefilled_except_last3(
        c, left, cursor, col_ws, rows, row_h,
        empty_cols_idx=(6,7,8),
        item_no_col_index=0
    )
    cursor -= ((1 + target_rows) * row_h + 4*mm)

    bottom_fields = [
        ({"key":"cashier_employee_number","label":"Cashier Employee Number"}, 0.50),
        ({"key":"recorder_employee_number","label":"Recorder Employee Number"}, 0.50),
    ]
    h_row = HEIGHTS["multi"]
    _draw_row(c, left, usable_w, max(cursor, m + h_row + 2*mm), h_row,
              bottom_fields, pad, fonts=label_font, sizes=label_font_sizes,
              value_font_max=value_font_max, prefill=prefill)

    c.save()
    try:
        if tmp_qr and os.path.exists(tmp_qr):
            os.remove(tmp_qr)
    except:
        pass
    return os.path.abspath(filename)

# -------- Example doc (preserves $date fields) --------
doc = {
  "id": "edfa2b5e-657e-4616-990b-1b69c983ee86",
  "previous_form_qr_code": "20251012",
  "current_form_qr_code": "87654321",
  "date": "2025-10-13",
  "location": "JEF Gas Station – Sikatuna Branch",
  "tin": "123-456-789",
  "branch": "JEF Biosciences Fuel Division",
  "created": {"$date": 1760457682654},
  "items": [
    {"item_number":"1","barcode":"4809992000101","type":"fuel","name":"Premium Gasoline","price":69.5,"unit":"liter","previous_quantity":570,"addstock":25.0,"sold":11.0,"current_quantity":584.0},
    {"item_number":"2","barcode":"4809992000102","type":"fuel","name":"Regular Gasoline","price":64.75,"unit":"liter","previous_quantity":760,"addstock":22.0,"sold":21.0,"current_quantity":761.0},
    {"item_number":"3","barcode":"4809992000103","type":"fuel","name":"Premium Diesel","price":61.2,"unit":"liter","previous_quantity":520,"addstock":0,"sold":0,"current_quantity":0},
    {"item_number":"4","barcode":"4809992000201","type":"lubricant","name":"Engine Oil SAE 40","price":250.0,"unit":"liter","previous_quantity":75,"addstock":0,"sold":0,"current_quantity":0},
    {"item_number":"5","barcode":"4809992000202","type":"lubricant","name":"Automatic Transmission Fluid (ATF)","price":310.0,"unit":"liter","previous_quantity":60,"addstock":0,"sold":0,"current_quantity":0},
    {"item_number":"6","barcode":"4809992000203","type":"lubricant","name":"Gear Oil EP 90","price":285.0,"unit":"liter","previous_quantity":55,"addstock":0,"sold":0,"current_quantity":0},
    {"item_number":"7","barcode":"4809992000204","type":"lubricant","name":"Hydraulic Oil ISO 68","price":295.0,"unit":"liter","previous_quantity":50,"addstock":0,"sold":0,"current_quantity":0},
    {"item_number":"8","barcode":"4809992000205","type":"lubricant","name":"2T Motorcycle Oil","price":180.0,"unit":"liter","previous_quantity":95,"addstock":10.0,"sold":20.0,"current_quantity":85.0}
  ],
  "is_empty": True
}

# Generate PDF (adds exactly 2 blank rows -> 10 rows total for 8 items)
pdf_path = render_empty_inventory_from_doc(
    doc,
    filename=os.path.join(os.getcwd(), "inventory_form_a4_empty.pdf"),
    page_size=A4,
    base_row_height_mm=4.6*1.2,
    qr_mm=16,
    qr_border=1
)

print("Empty inventory form PDF generated at:", pdf_path)


Empty inventory form PDF generated at: h:\github4\jefstore-gasstations-backend\reportlab\inventories\empty\inventory_form_a4_empty.pdf
